### Selected Ngram Repeat With Youtube Link

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

In [4]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [5]:
#data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Deployment3/Data/Manuel Select"

In [6]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/4-Selected Ngram Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

##### Link Count

In [28]:
# file extention
file_ext = "Twogram"

In [29]:
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Result/Ngram Youtube Link Manuel"
file = f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_12.xlsx"   ########*****
#sheet = "Sheet1"  # Sheet1

In [30]:
#df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}", sheet_name=f"{sheet}")
df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}")
df_link_selected_manuel

,video_id,start_time,end_time,text
0,NtoRI5nlwKw,1073,1075,duyuyor musun
1,CwuGepQDBrM,153,155,güzel bir
2,UQDUQnVUtq4,942,944,güzel bir
3,dRCz3FhBk08,97,99,güzel bir
4,7XVwNBlTIWs,13574,13576,güzel bir
...,...,...,...,...
973,y7o33BX7ec0,428,430,yardım etmek
974,0qGrmkm6r44,520,522,yardım etmek
975,Chvvzy8loOE,1369,1371,yardım etmek
976,0Hyrz58zvcA,1400,1402,yardım etmek


In [31]:
# disabled because of youtube tools
#df_link_selected_manuel["video_url"] = "https://www.youtube.com/watch?v="+df_link_selected_manuel['video_id'].map(str)+"&t="+df_link_selected_manuel['start_time'].map(str)+"s"
#df_link_selected_manuel

In [32]:
df_count = df_link_selected_manuel.groupby(["text"])[["text"]].count()
df_count.rename(columns={"text":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,text,count
0,ne yapabilirim,14
1,ne olduğunu,10
2,öyle bir,10
3,kimse yok,9
4,ne tür,9
...,...,...
173,yok mu,3
174,şey yok,3
175,beni duyuyor,2
176,neydi böyle,1


In [33]:
df_count = df_count[df_count["count"] > 2]

In [34]:
len(df_count)

175

In [35]:
df_count_merge = pd.merge(df_link_selected_manuel, df_count, how="inner",on= "text")
df_count_merge.sort_values(by=["count","text"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,video_id,start_time,end_time,text,count
0,Jfh5yVT4OPk,418,420,ne yapabilirim,14
1,T5krOQidV5s,714,716,ne yapabilirim,14
2,vGkcBnwz8UQ,1,3,ne yapabilirim,14
3,c90800j_ED4,1066,1068,ne yapabilirim,14
4,MaSTWrRUUfY,843,845,ne yapabilirim,14
...,...,...,...,...,...
969,m3Clq7uulPg,4093,4095,bir bak,3
970,GQgS2ruEJAs,5182,5184,bir bak,3
971,n2t0ueWtUNI,54,56,başka bir,3
972,N0a3f_R2BfQ,790,792,başka bir,3


In [122]:
# ****** ???? edit path
df_count.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 6/Data/\
Deployment/Talk Time 6 {file_ext} Video List.xlsx", index=False)

In [123]:
df_count_merge.to_excel(f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_Count.xlsx", index=False)   ########*****

##### Link Repeat

In [124]:
# ******  
path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 6/Result/\
2-Find Word Group Youtube Link/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_Count.xlsx"   ########*****
#sheet2 = "Sheet1"  # Sheet1

In [125]:
#df_word_group_time_loc = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
df_word_group_time_loc = pd.read_excel(f"{path_folder_file}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,video_id,start_time,end_time,text,video_url,count
0,V9n8LpTPcfM,1002,1005,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=V9n8LpTPcfM&t=...,7
1,hM5s4liNn7g,830,832,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=hM5s4liNn7g&t=...,7
2,ao2kpL29HwI,454,457,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=ao2kpL29HwI&t=...,7
3,j1jgprjyYxw,815,818,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=j1jgprjyYxw&t=...,7
4,miIgOGcO2tY,649,651,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=miIgOGcO2tY&t=...,7
5,kQkG6D4c_xU,1077,1079,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=kQkG6D4c_xU&t=...,7
6,8ek8uhoEQ9w,662,664,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=8ek8uhoEQ9w&t=...,7
7,Odbd1stkcRk,6456,6458,bir iyi bir de kötü haberim var,https://www.youtube.com/watch?v=Odbd1stkcRk&t=...,2
8,O3Xde6lnWqw,349,351,bir iyi bir de kötü haberim var,https://www.youtube.com/watch?v=O3Xde6lnWqw&t=...,2


In [126]:
search_list = df_word_group_time_loc.loc[:,"text"].unique()
len(search_list)

2

In [127]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["text","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,text,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [128]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [129]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["text"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result["web_format"] = f"[play_lms_videos video_id="+df_all_join_result["video_id"].map(str)+" start_\
time="+df_all_join_result["start_time"].map(str)+" end_time="+df_all_join_result["end_time"].map(str)+"]"
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)

In [130]:
df_all_join_result

,video_id,start_time,end_time,text,count,web_format
0,"V9n8LpTPcfM,hM5s4liNn7g,ao2kpL29HwI,j1jgprjyYx...","1002,830,454,815,649,1077,662,815,5,815,5,815,5","1005,832,457,818,651,1079,664,818,8,818,8,818,8",söylemek istediğin başka bir şey var mı,7.0,"[play_lms_videos video_id=V9n8LpTPcfM,hM5s4liN..."
1,"Odbd1stkcRk,O3Xde6lnWqw,O3Xde6lnWqw,Q-8I-uMUMY...","6456,349,349,5,349,5,349,5","6458,351,351,8,351,8,351,8",bir iyi bir de kötü haberim var,2.0,"[play_lms_videos video_id=Odbd1stkcRk,O3Xde6ln..."


#### Copy Move And Delete

In [131]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_*.xlsx")
output_file

['Turkish_40000_Word_Youtube_Talk_Time_Join_Sevengram.xlsx']

In [132]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [133]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass